In [33]:
import os
from dotenv import load_dotenv
from typing import Literal
from pprint import pprint
from langchain import hub
from typing import Literal,List
from typing_extensions import TypedDict
from langchain_cohere import CohereEmbeddings, ChatCohere
from langgraph.graph import END,START,StateGraph
from langchain.schema import Document
from langchain_core.output_parsers import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings,ChatOpenAI
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import Field,BaseModel
from langchain_community.tools.tavily_search import TavilySearchResults



load_dotenv()
langchain_api = os.getenv('LANGCHAIN_API_KEY')
TAVILY_API_KEY = os.getenv('TAVILY_API_KEY')
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
COHERE_API_KEY=os.getenv('COHERE_API_KEY')

urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]

### Index

In [34]:
docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

embd = CohereEmbeddings(model="embed-multilingual-v3.0")

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=512,chunk_overlap=0
)
splited_text = text_splitter.split_documents(docs_list)

vectorspace = Chroma.from_documents(
    embedding=embd,
    documents=splited_text,
)

retriever = vectorspace.as_retriever()


### Router

In [35]:
class web_search(BaseModel):
    """
    The internet. Use web_search for questions that are related to anything else than agents, prompt engineering, and adversarial attacks.
    """
    query: str=Field(description="The query to use when searching the internet")

class vectorstore(BaseModel):
    """
    A vectorstore containing documents related to agents, prompt engineering, and adversarial attacks. Use the vectorstore for questions on these topics.   
    """
    query: str=Field(description= "The query to use when searching the vectorstore")


preamble = """You are an expert at routing a user question to a vectorstore or web search.
The vectorstore contains documents related to agents, prompt engineering, and adversarial attacks.
Use the vectorstore for questions on these topics. Otherwise, use web-search."""

llm = ChatCohere(model="command-a-03-2025",temperature=0)
structured_llm_router = llm.bind_tools(tools=[web_search,vectorstore],
                                       preamble=preamble)

route_prompt = ChatPromptTemplate(
    [
        ('user', "{question}")
    ]
)

question_router = route_prompt|structured_llm_router

response = question_router.invoke(
    {"question": "Who will the Bears draft first in the NFL draft?"}
)


In [36]:
response.additional_kwargs['tool_calls'][0]['function']['name']

'web_search'

### Retireval Grader

In [62]:
# class GradeDocuments(BaseModel):
#     binary_score: Literal["yes","no"] = Field(
#         description="Documents are relevant to the question: 'yes' or 'no'"
#     )

# preamble = """You are a grader assessing relevance of a retrieved document to a user question. \n
# If the document contains keyword(s) or semantic meaning related to the user question, grade it as relevant. \n
# Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question."""


# llm = ChatCohere(model="command-a-03-2025",temperature=0)
# structure_llm_grader = llm.with_structured_output(GradeDocuments,preamble=preamble)
# grader_prompt = ChatPromptTemplate.from_messages([
#     ('human',"Retrieved Document: \n\n{documents} \n\n User question: \n\n {question}"),
# ])
class GradeDocuments(BaseModel):
    binary_score : Literal["yes","no"] = Field(
        description="Documents are relevant to the question, 'yes', or 'no'"
    )

llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)
structured_llm_grader = llm.with_structured_output(GradeDocuments)

system = """You are a grader assessing relevance of a retrieved document to a user question. \n 
    If the document contains keyword(s) or semantic meaning related to the question, grade it as relevant. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question."""
grader_prompt = ChatPromptTemplate.from_messages(
    [
        ("system" , system),
        ("user", "Retrieved document: \n\n {document} \n\n User question: {question}")
    ]
)

retrieval_grader_chain = grader_prompt|structured_llm_grader
question = "Who will the Bears draft first in the NFL draft?"
docs = retriever.get_relevant_documents(question)[0].page_content
generation = retrieval_grader_chain.invoke({"document": docs, "question": question})
print(generation)

/opt/anaconda3/envs/pytorch/lib/python3.12/site-packages/langchain_openai/chat_models/base.py:1900: UserWarning: Received a Pydantic BaseModel V1 schema. This is not supported by method="json_schema". Please use method="function_calling" or specify schema via JSON Schema or Pydantic V2 BaseModel. Overriding to method="function_calling".
  warnings.warn(


binary_score='no'
